In [1]:
import pandas as pd
#data collection
hour_data=pd.read_csv("./hourly_data.csv")
week_data=pd.read_csv("./weekly_data.csv")
month_data=pd.read_csv("./monthly_data.csv")

In [2]:
import datetime as dt
#data cleaning and manipulation steps
month_data['year'] = month_data['date'].apply(lambda x: int(x.split('-')[0]))
month_data['month'] = month_data['date'].apply(lambda x: int(x.split('-')[1]))
month_data['day'] = month_data['date'].apply(lambda x: int(x.split('-')[2]))

week_data['year'] = week_data['date'].apply(lambda x: int(x.split('-')[0]))
week_data['month'] = week_data['date'].apply(lambda x: int(x.split('-')[1]))
week_data['day'] = week_data['date'].apply(lambda x: int(x.split('-')[2]))

hour_data["time_part"]=hour_data["date"].apply(lambda x: (x.split(' ')[1]))
hour_data["date_part"]=hour_data["date"].apply(lambda x: (x.split(' ')[0]))
hour_data['year'] = hour_data['date_part'].apply(lambda x: int(x.split('-')[0]))
hour_data['month'] = hour_data['date_part'].apply(lambda x: int(x.split('-')[1]))
hour_data['day'] = hour_data['date_part'].apply(lambda x: int(x.split('-')[2]))
hour_data['hour'] = hour_data['time_part'].apply(lambda x: int(x.split(':')[0]))
hour_data['min'] = hour_data['time_part'].apply(lambda x: int(x.split(':')[1]))
hour_data['second'] = hour_data['time_part'].apply(lambda x: int(x.split(':')[2]))

In [3]:
#Extracting more information
import datetime as dt
import math
week_data["date"] = pd.to_datetime(week_data["date"])
week_data['week_in_month'] = pd.to_numeric(week_data.date.dt.day/7)
week_data['week_in_month'] = week_data['week_in_month'].apply(lambda x: math.ceil(x))
hour_data["date_part"] = pd.to_datetime(hour_data["date_part"])
hour_data['week_in_month'] = pd.to_numeric(hour_data.date_part.dt.day/7)
hour_data['week_in_month'] = hour_data['week_in_month'].apply(lambda x: math.ceil(x))
#dropping 2022 8th month week 4 data as that data is not availbale in week_data
hour_data=hour_data.drop(hour_data[(hour_data['year'] == 2022) &
                                   (hour_data['month'] == 8) & (hour_data['week_in_month'] == 4)].index)


In [4]:
#merging week and hour data
merged_hour_week=pd.merge(hour_data, week_data,  how='outer', left_on=['year','month','week_in_month'], 
         right_on = ['year','month','week_in_month'])

In [5]:
merged_hour_week

,time_hour,value_hour,date_x,time_part,date_part,year,month,day_x,hour,min,second,week_in_month,time_week,value_week,date_y,day_y
0,1483228800,30,2017-01-01 00:00:00,00:00:00,2017-01-01,2017,1,1,0,0,0,1,1.483229e+09,100.0,2017-01-01,1.0
1,1483232400,34,2017-01-01 01:00:00,01:00:00,2017-01-01,2017,1,1,1,0,0,1,1.483229e+09,100.0,2017-01-01,1.0
2,1483236000,33,2017-01-01 02:00:00,02:00:00,2017-01-01,2017,1,1,2,0,0,1,1.483229e+09,100.0,2017-01-01,1.0
3,1483239600,43,2017-01-01 03:00:00,03:00:00,2017-01-01,2017,1,1,3,0,0,1,1.483229e+09,100.0,2017-01-01,1.0
4,1483243200,32,2017-01-01 04:00:00,04:00:00,2017-01-01,2017,1,1,4,0,0,1,1.483229e+09,100.0,2017-01-01,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49400,1661108400,90,2022-08-21 19:00:00,19:00:00,2022-08-21,2022,8,21,19,0,0,3,1.661040e+09,31.0,2022-08-21,21.0
49401,1661112000,91,2022-08-21 20:00:00,20:00:00,2022-08-21,2022,8,21,20,0,0,3,1.661040e+09,31.0,2022-08-21,21.0
49402,1661115600,96,2022-08-21 21:00:00,21:00:00,2022-08-21,2022,8,21,21,0,0,3,1.661040e+09,31.0,2022-08-21,21.0
49403,1661119200,94,2022-08-21 22:00:00,22:00:00,2022-08-21,2022,8,21,22,0,0,3,1.661040e+09,31.0,2022-08-21,21.0


In [6]:
import math
for i,row in merged_hour_week.iterrows():
    if math.isnan(row["value_week"]):
        if row["month"]!=12:
            merged_hour_week.at[i,"value_week"]=((merged_hour_week[(merged_hour_week["year"]==row["year"])&(merged_hour_week["month"]==row["month"]+1)&(merged_hour_week["week_in_month"]==1)]["value_week"].values[0]))
        else:
            merged_hour_week.at[i,"value_week"]=((merged_hour_week[(merged_hour_week["year"]==row["year"]+1)&(merged_hour_week["month"]==1)&(merged_hour_week["week_in_month"]==1)]["value_week"].values[0]))
        
        

In [7]:
merged_hour_week["year_month_weekOfMonth"]=(merged_hour_week['year'].map(str) + 
                                     merged_hour_week['month'].map(str)+
                                     merged_hour_week['week_in_month'].map(str))
computed_mean_year_month_week_df=(merged_hour_week.groupby('year_month_weekOfMonth')
                             ['value_hour'].mean().to_frame().reset_index())
computed_mean_year_month_week_df=(computed_mean_year_month_week_df.rename
                             (columns={"value_hour": "computed_mean_value_hour"}))
merged_hour_week=(pd.merge(merged_hour_week, computed_mean_year_month_week_df, 
                      how='left', left_on=['year_month_weekOfMonth'], 
                         right_on = ['year_month_weekOfMonth']))


In [8]:
#computinh value_hour:computed_weekly_avg::(new_value):value_week
merged_hour_week["converted_weekly_value_hour"]=((merged_hour_week["value_hour"]
                                                  *merged_hour_week["value_week"])
                                                /merged_hour_week["computed_mean_value_hour"])


In [9]:
month_data_short=month_data[["year","month","value_month"]]
merged_hour_week_month=pd.merge(merged_hour_week, month_data_short,  how='left', left_on=['year','month'], 
         right_on = ['year','month'])
merged_hour_week_month["converted_monthly_value_avg"]=((merged_hour_week_month["converted_weekly_value_hour"]
                                       *merged_hour_week_month["value_month"])
                                      /merged_hour_week_month["value_week"])

In [21]:
final_df=(merged_hour_week_month[["time_hour","date_x","value_hour","converted_monthly_value_avg","value_month"]])

In [22]:
final_df=(final_df.rename(columns={"date_x": "date",
                                   "converted_monthly_value_avg":"new_value_hour"}))

In [23]:
final_df

,time_hour,date,value_hour,new_value_hour,value_month
0,1483228800,2017-01-01 00:00:00,30,2.892949,6
1,1483232400,2017-01-01 01:00:00,34,3.278676,6
2,1483236000,2017-01-01 02:00:00,33,3.182244,6
3,1483239600,2017-01-01 03:00:00,43,4.146561,6
4,1483243200,2017-01-01 04:00:00,32,3.085813,6
...,...,...,...,...,...
49400,1661108400,2022-08-21 19:00:00,90,27.838617,23
49401,1661112000,2022-08-21 20:00:00,91,28.147935,23
49402,1661115600,2022-08-21 21:00:00,96,29.694524,23
49403,1661119200,2022-08-21 22:00:00,94,29.075889,23
